In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional,GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

tfds = tf.data.Dataset

In [3]:
cd drive/MyDrive/datasets/Bengali_poets

/content/drive/MyDrive/datasets/Bengali_poets


In [4]:
ls

ishwarchandragupta.txt  kaminiroy.txt    poets.csv         satyendranath.txt
jatindramohan.txt       madhusudan.txt   rabindranath.txt  sukanta.txt
jibanananda.txt         nazrulislam.txt  README.md         sukumar.txt


In [5]:
import glob

my_files_path = glob.glob(r'*.txt')
print(my_files_path)
text=''
for txt_file in my_files_path[:2] :
    text += open(txt_file, 'r',encoding='utf-8',
                    errors='ignore').read().replace(u'\xa0', u' ')
    text += '\n\n\n'
    
# print(text)

['madhusudan.txt', 'nazrulislam.txt', 'ishwarchandragupta.txt', 'jatindramohan.txt', 'rabindranath.txt', 'satyendranath.txt', 'jibanananda.txt', 'sukanta.txt', 'kaminiroy.txt', 'sukumar.txt']


In [6]:
text

'\nকাশীরাম দাস\n\n\nচন্দ্রচূড় জটাজালে আছিলা যেমতি\nজাহ্নবী, ভারত-রস ঋষি দ্বৈপায়ন, \nঢালি সংস্কৃত-হ্রদে রাখিলা তেমতি; — \nতৃষ্ণায় আকুল বঙ্গ করিত রোদন। \nকঠোরে গঙ্গায় পূজি ভগীরথ ব্রতী, \n(সুধন্য তাপস ভবে, নর-কুল-ধন! ) \nসগর-বংশের যথা সাধিলা মুকতি, \nপবিত্ৰিলা আনি মায়ে, এ তিন ভুবন; \nসেই রূপে ভাষা-পথ খননি স্ববলে, \nভারত-রসের স্রোতঃ আনিয়াছ তুমি \nজুড়াতে গৌড়ের তৃষা সে বিমল জলে। \nনারিবে শোধিতে ধার কভু গৌড়ভূমি। \nমহাভারতের কথা অমৃত-সমান। \nহে কাশি, কবীশদলে তুমি পুণ্যবান্॥(চতুদ্দর্শপদী কবিতাবলী)\n\n\n\n\nপৃথিবী\n\n\n১           হে বসুধে, জগৎজননি! \n     দয়াময়ী তুমি, সতি, বিদিত ভুবনে!\n           যবে দশানন অরি,\n     বিসৰ্জ্জিলা হুতাশনে জানকী সুন্দরী,\n           তুমি গো রাখিলা বরাননে।\nতুমি, ধনি, দ্বিধা হয়ে,       বৈদেহীরে কোলে লয়ে,\n    জুড়ালে তাহার জ্বালা বাসুকি-রমণি!২           হে বসুধে, রাধা বিরহিণী!\n    তার প্রতি আজি তুমি বাম কি কারণে?\nশ্যামের বিরহানলে,           সুভগে, অভাগা জ্বলে,\n          তারে যে কর না তুমি মনে?\nপুড়িছে অবলা বালা,        কে সম্বরে তার জ্বালা,\n    হা

In [7]:
letters = sorted(set(text))
print(letters)
vocab = len(letters)
vocab

['\t', '\n', ' ', '!', '"', "'", '(', ')', '*', ',', '-', '.', '/', '2', ':', ';', '=', '?', 'M', '[', ']', '_', '`', 'a', 'f', 'l', 'o', 'r', 's', 't', 'y', '|', '~', '\x9d', 'ž', '।', '॥', 'ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ড়', 'ঢ়', 'য়', '০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯', 'ৰ', '৷', '\u200c', '\u200d', '‐', '–', '—', '‘', '’', '“', '”', '…', '−']


122

In [8]:
char_to_ind = {x:i+1 for i,x in enumerate(letters)}
ind_to_char = {i+1:x for i,x in enumerate(letters)}
encoded_text = np.array([char_to_ind[c] for c in text])
encoded_text

array([ 2, 52, 85, ...,  2,  2,  2])

In [ ]:
# char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)
n_chars = len(text)
seq_length = max([len(x) for x in text.split('\n')])
dataX, dataY = [],[]

for i in range(0, n_chars - seq_length, 1):  #step=1 means each sentence is offset just by a single letter
    dataX.append(encoded_text[ i : i + seq_length ])  #Sequence in
    dataY.append(tf.one_hot(encoded_text[ i + seq_length ], vocab))  #Sequence out
dataX = np.array(dataX)
dataY = np.array(dataY)

In [ ]:
# seq_length = max([len(x) for x in text.split('\n')])
# print(seq_length)
# n_chars = len(text)

# dataX = []
# dataY = []
# for i in range(0, n_chars - seq_length, 1):
# 	seq_in = encoded_text[i:i + seq_length]
# 	seq_out = encoded_text[i + seq_length]
# 	dataX.append(seq_in)
# 	dataY.append(seq_out)
# n_patterns = len(dataX)

# dataY = tf.one_hot(dataY, vocab)
# dataX = np.reshape(dataX, (len(dataX), seq_length, 1))
print(seq_length)

In [ ]:
min([len(x) for x in dataX])

In [ ]:
# # pad sequences 

# input_sequences =  np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
# # np.random.shuffle(input_sequences)
# # input_sequences1 = tf.data.Dataset.from_tensor_slices(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
# print(input_sequences)
# # create predictors and label
# def predictor_and_label(sequences):
#     return sequences[:-1], tf.one_hot(sequences[-1],total_words)


# # dataset = input_sequences.map(lambda x: predictor_and_label(x))
# # dataset

In [ ]:
print(seq_length)
model = Sequential()
model.add(Embedding(vocab, 128, input_length=seq_length))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256))
model.add(Dense(128, activation='relu'))
model.add(Dense(vocab, activation='softmax'))
model.summary()


In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
history = model.fit(dataX, dataY, batch_size = 32,epochs=100, verbose=0)

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

In [ ]:
plot_graphs(history,'accuracy')

In [ ]:
seed_text = "\nকে\n\n\nবল দেখি এ জগতে ধার্মিক কে হয়,\nসর্ব জীবে দয়া যার, ধার্মিক সে হয়।\nবল দেখি এ জগতে সুখী বলি কারে,\nসতত আরোগী যেই, সুখী বলি তারে।\nবল দেখি এ জগতে বিজ্ঞ বলি কারে,\nহিতাহিত বোধ যার, বিজ্ঞ বলি তারে।\nবল দেখি এ জগতে ধীর বলি কারে,\nবিপদে যে স্থির থাকে, ধীর বলি তারে।\nবল দেখি এ জগতে মূর্খ বলি কারে,\nনিজ কার্য "
next_words = 500
prev = -1
c= 0
for _ in range(next_words):
    token_list = [char_to_ind[x] for x in seed_text]
    t_list = pad_sequences([token_list], maxlen=seq_length, padding='pre')
    pred = model.predict(t_list)
    output_char = ind_to_char[np.argmax(pred)]
    print(output_char)
    seed_text += output_char
print(seed_text)

In [ ]:
# মিছা কেন কুল নিয়া কর আঁটাআঁটি। মানিতে তোষে। আর তোষে। ধরে ভক্ষণ ভক্ষণ অনেকেই ধরে রাখে আবা আবা ধরে ধরে হরণ হরণ আর আর তাজা তাজা। তাজা। বুক ঠাই অনেকেই রাখে রাখে রাখে রাখে রাখে রাখে ধরে ধরে ঠাই ধরে ধরে রাখে অনেকেই জগতে বলি হয় চায় ধন। ধরে ধরে ধরে ধরে ধরে ধরে রাখে অনেকেই রাখে সে আর চায় ধন। ধরে ধরে ধরে ধরে ধরে ধরে অনেকেই রাখে বলি আবা চায় ধন। ধরে ধরে ধরে ধরে ধরে ধরে অনেকেই রাখে সে আর চায় ধন। ধরে ধরে ধরে ধরে ধরে ধরে অনেকেই রাখে বলি আবা চায় ধন। ধরে ধরে ধরে ধরে ধরে ধরে

# মিছা কেন কুল নিয়া কর আঁটাআঁটি।',
#  'এ যে কুল কুল নয় সার মাত্র আঁটি।।',
#  'কুলের গৌরব কর কোন্ অভিমানে।',
#  'মূলের হইলে দোষ কেবা তারে মানে।।',
#  'ঘটকের মুখে সুধু কুলীনের চোপা।',
#  'রস নাই যশ কিসে কুল হল টোপা।।',
#  'আদর হইত তবে ভাঙ্গিলে অরুচি।',
#  'পোকাধরা সোঁকা ভার দেখে যায় রুচি।।',
#  'অতএব বৃথা এই কুলের আচার।',
#  'ইথে নাহি রক্ষা পায় কুলের আচার।।',
#  'কুলের সম্ভ্রম বল করিব কেমনে।',
#  'শতেক বিধবা হয় একেক মরণে।।',
#  'বগলেতে বৃষকাষ্ঠ শক্তিহীন যেই।',
#  'কোলের কুমারী লয়ে বিয়ে করে সেই।।',
#  'দুধে দাঁত ভাঙ্গে নাই শিশু নাম যার।',
#  'পিতামহী সম নারী দারা হয় তার।।',
#  'নরনারী তুল্য বিনা কিসে মন তোষে।',
#  'ব্যভিচার হয় শুদ্ধ এই সব দোষে।।',
#  'কুলকল্পে নয় রূপ সুলক্ষণ যাহা।',
#  'অবশ্য প্রামাণ্য করি শিরোধার্য তাহা।।',
#  'নচেৎ যে কুল তাহা দোষের কারণ।',
#  'পাপের গৌরব কেন করিব ধারণ।।',
#  'হে বিভু করুণাময় বিনয় আমার।',
#  'এ দেশের কুলধর্ম করহ সংহার।।'